In [5]:

%%shell
pip install -q cython
pip install -qU 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

pip install gdown
gdown --id 1HGnvRZW-HWkyh7x1_ZlvSAPDcx1OGlNh
gdown --id 1_HrcPnL5HnmG3m6XvQRK2IYAHf5a5mS_
unzip few_data_split.zip

git clone https://github.com/comptech-winter-school/coal-composition-control -b dev


  Preparing metadata (setup.py) ... done
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1HGnvRZW-HWkyh7x1_ZlvSAPDcx1OGlNh

but Gdown can't. Please check connections and permissions.
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_HrcPnL5HnmG3m6XvQRK2IYAHf5a5mS_
To: /content/example.png
100% 1.16M/1.16M [00:00<00:00, 119MB/s]
unzip:  cannot find 

In [6]:
from packaging import version
import numpy as np

if version.parse(np.__version__) >= version.parse("1.24.0"):
  np.float = np.float64

In [7]:
%cd coal-composition-control
!pip install -qU numpy
!pip install -r requirements.txt

/content/coal-composition-control
ERROR: Could not find a version that satisfies the requirement torchvision~=0.11.1 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2)
ERROR: No matching distribution found for torchvision~=0.11.1


In [8]:
import torch
from PIL import Image
from pathlib import Path
from train.instance_segmentation.mask_rcnn_dataset import CoalFractionDataset

In [9]:
import os
import random

def seed_everything(SEED=411, seed_all=False) -> None:
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED']=str(SEED)
    if seed_all:
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.benchmark = False

In [10]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [11]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.11.0


cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 459044, done.
remote: Counting objects: 100% (39503/39503), done.
remote: Compressing objects: 100% (1826/1826), done.
remote: Total 459044 (delta 37668), reused 39399 (delta 37618), pack-reused 419541
Receiving objects: 100% (459044/459044), 904.51 MiB | 28.02 MiB/s, done.
Resolving deltas: 100% (426767/426767), done.
Note: switching to 'v0.11.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 58a60b2eb4 Fixing bug on SSD backbone freezing. (#4591)


In [12]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(p=0.2))
        transforms.append(T.RandomPhotometricDistort(p=0.1))

    return T.Compose(transforms)

In [13]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

dataset = CoalFractionDataset(
    root='/content/drive/MyDrive/dataset',
    vgg_json='/content/drive/MyDrive/via_project_25Jan2024_1h34m_json.json',
    width=2000,
    height=1500,
    transforms=get_transform(train=True)
    )
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn
)
# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 98.4MB/s]


In [14]:
# use our dataset and defined transformations
dataset = CoalFractionDataset(
    root='/content/drive/MyDrive/dataset',
    vgg_json='/content/drive/MyDrive/via_project_25Jan2024_1h34m_json.json',
    width=2000,
    height=1500,
    transforms=get_transform(train=True)
    )
dataset_test = CoalFractionDataset(
    root='/content/drive/MyDrive/dataset_test',
    vgg_json='/content/drive/MyDrive/via_project_25Jan2024_1h18m_json.json',
    width=3000,
    height=2500,
    transforms=get_transform(train=False)
    )


# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

In [15]:
from google.colab.patches import cv2_imshow
print(dataset.masks['00a2145de1886cb9eb88869c85d74080.dicom.jpg'])
print(dataset.size)
blank = np.zeros((2000, 1500))


[((219, 1154), (224, 1072), (251, 1002), (301, 942), (383, 899), (455, 875), (540, 858), (614, 848), (687, 843), (783, 853), (860, 872), (920, 894), (969, 928), (945, 1014), (913, 988), (872, 942), (819, 920), (757, 911), (672, 906), (595, 913), (479, 935), (393, 964), (311, 1005), (279, 1055), (260, 1096), (265, 1111), (219, 1154))]
{'width': 2000, 'height': 1500}


In [16]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 121MB/s]


In [17]:
import shutil
fol_path = '/content/drive/MyDrive/dataset/.ipynb_checkpoints'
if os.path.exists(fol_path):
  shutil.rmtree(fol_path)

fold_path = '/content/drive/MyDrive/dataset_test/.ipynb_checkpoints'
if os.path.exists(fold_path):
  shutil.rmtree(fold_path)

In [ ]:
from torch.optim.lr_scheduler import StepLR
num_epochs = 15

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=2)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [0/8]  eta: 0:06:41  lr: 0.000719  loss: 2.7232 (2.7232)  loss_classifier: 0.5488 (0.5488)  loss_box_reg: 0.0339 (0.0339)  loss_mask: 2.0141 (2.0141)  loss_objectness: 0.1229 (0.1229)  loss_rpn_box_reg: 0.0035 (0.0035)  time: 50.2444  data: 3.6214
Epoch: [0]  [2/8]  eta: 0:04:28  lr: 0.002146  loss: 2.7232 (2.6421)  loss_classifier: 0.5488 (0.5200)  loss_box_reg: 0.0339 (0.0288)  loss_mask: 2.0141 (1.8820)  loss_objectness: 0.2031 (0.1994)  loss_rpn_box_reg: 0.0162 (0.0120)  time: 44.8217  data: 1.2160
Epoch: [0]  [4/8]  eta: 0:02:59  lr: 0.003573  loss: 1.7531 (1.9327)  loss_classifier: 0.4234 (0.3786)  loss_box_reg: 0.0342 (0.0539)  loss_mask: 1.0761 (1.3445)  loss_objectness: 0.1229 (0.1465)  loss_rpn_box_reg: 0.0086 (0.0093)  time: 44.7570  data: 0.7316
Epoch: [0]  [6/8]  eta: 0:01:29  lr: 0.005000  loss: 1.7531 (2.2047)  loss_classifier: 0.2005 (0.2846)  loss_box_reg: 0.0342 (0.0485)  loss_mask: 1.0761 (1.7289)  loss_objectness: 0.1068 (0.1337)  loss_rpn_box_reg: 0.008

In [ ]:
model.eval()
model.roi_heads.score_thresh=0.5
model.roi_heads.nms_thresh=0.2

In [ ]:
torch.save(model, '/content/sample_data/mask-rcnn.pth')
# model = torch.load('/content/mask-rcnn.pth')

In [ ]:
# pick one image from the test set
# img, _ = dataset_test[0]
to_tensor = T.ToTensor()
img, _ = to_tensor(Image.open('/content/drive/MyDrive/333.jpg').convert("RGB"))
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
prediction[0]['masks'].shape

In [ ]:
import torchvision.transforms as transforms
from torchvision.utils import draw_segmentation_masks, draw_bounding_boxes

to_pil_image = transforms.ToPILImage()
byte_img = img.mul(255).byte()

In [ ]:
def mask_on_image(byte_img, prediction, th=0.7):
  masks = torch.squeeze(prediction[0]['masks']).cpu()
  threshold = torch.Tensor([th])
  masks = (masks > threshold).bool()
  masks_on_image = draw_segmentation_masks(
      image = byte_img,
      masks = masks,
      alpha=0.6
      )
  return to_pil_image(masks_on_image)

In [ ]:
def box_on_image(byte_img, prediction):
  boxes = torch.squeeze(prediction[0]['boxes'])
  boxes_on_image = draw_bounding_boxes(
      image=byte_img,
      boxes=boxes,
      width=4
      )
  return to_pil_image(boxes_on_image)

In [ ]:
to_pil_image(byte_img)

In [ ]:
box_on_image(byte_img, prediction)

In [ ]:
mask_on_image(byte_img, prediction, th=0.9)
mask_on_image(byte_img, prediction, th=0.8)
mask_on_image(byte_img, prediction, th=0.7)
mask_on_image(byte_img, prediction, th=0.6)
mask_on_image(byte_img, prediction, th=0.5)
mask_on_image(byte_img, prediction, th=0.4)
mask_on_image(byte_img, prediction, th=0.3)
mask_on_image(byte_img, prediction, th=0.2)
mask_on_image(byte_img, prediction, th=0.1)

In [ ]:
mask_on_image(byte_img, prediction, th=0.1)

In [ ]:
!rm -rf find -type d -name .ipynb_checkpoints